In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import astropy.units as u
import astropy.constants as uc

from ipywidgets import interact
import ipywidgets as widgets

plt.style.use("nonos.default")

from kintomo.api import Sculpture, Shape, Velocity, Grid, Geometry, projection_deposition_visible

In [25]:
directory = "../data"
from nonos.api import GasDataSet
ds = GasDataSet(1, directory=directory)
unit_l = 93.0*u.au
unit_m = 1.2*u.M_sun
unit_v = np.sqrt(uc.G*unit_m/unit_l).to(u.km/u.s)

pps = np.linspace(0, 2*np.pi, 150)

# field = "DUST3_"
field = ""

RHO = ds[f"{field}RHO"].data
RHO = np.repeat(RHO[:,:,None], len(pps), axis=2)
RHOF = RHO.flatten()
rho_mask = (RHOF>1e-7)
print(f"removing {100*abs(len(RHOF)-len(RHOF[rho_mask]))/len(RHOF):.1f}% of the point cloud")

rrs, tts = (ds.coords.get_axis_array_med(kk) for kk in ("r", "theta"))
RR, TT, PP = np.meshgrid(rrs, tts, pps, indexing="ij")
RRF = RR.flatten()
TTF = TT.flatten()
PPF = PP.flatten()

x0 = RRF*np.sin(TTF)*np.cos(PPF)
y0 = RRF*np.sin(TTF)*np.sin(PPF)
z0 = RRF*np.cos(TTF)
sculpture = Sculpture(x=x0[rho_mask], y=y0[rho_mask], z=z0[rho_mask]).to(unit=unit_l)

r, theta, phi = sculpture.spherical_coordinates()

VR = ds[f"{field}VX1"].data
VT = ds[f"{field}VX2"].data
VP = ds[f"{field}VX3"].data

VR = np.repeat(VR[:,:,None], len(pps), axis=2)
VT = np.repeat(VT[:,:,None], len(pps), axis=2)
VP = np.repeat(VP[:,:,None], len(pps), axis=2)

VRF = VR.flatten()
VTF = VT.flatten()
VPF = VP.flatten()

velocity = Velocity(
    geometry=Geometry("spherical"),
    v1=VRF[rho_mask],
    v2=VTF[rho_mask],
    v3=VPF[rho_mask],
).to_cartesian(phi=phi, theta=theta).to(unit=unit_v)

# nx, ny, nz = (65, 65, 33)
nx, ny, nz = (255, 255, 45)
grid = Grid.encompass(
    sculpture=sculpture, 
    dimension=(nx, ny, nz),
)

removing 29.0% of the point cloud


In [26]:
vshift_extent = velocity.shift(get="mean")
dv = 0.02*vshift_extent

def plot_vmap(inclination, vshift):
    v_visible = projection_deposition_visible(
        grid=grid,
        sculpture=sculpture,
        velocity=velocity,
        angle=inclination,
        method="ngp",
    )

    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.pcolormesh(
        "x",
        "y",
        v_visible.T,
        data=grid.cell_edges,
        vmin=-vshift_extent, 
        vmax=vshift_extent,
        cmap="RdBu_r",
    )
    contours = ax.contour(
        "x", 
        "y", 
        v_visible.T, 
        data=grid.cell_centers, 
        levels=np.linspace(vshift-dv,vshift+dv+0.003,10), 
        colors="k", 
        linestyles="solid"
    )
    ax.set(
        aspect="equal", 
        ylim=((3/5)*grid.yedge.min(), (3/5)*grid.yedge.max()), 
        xlabel=f"x [{sculpture.get_unit}]", 
        ylabel=f"y [{sculpture.get_unit}]"
    )

    from mpl_toolkits.axes_grid1 import make_axes_locatable

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(
        im, cax=cax, orientation="vertical"
    )
    cbar.set_label(rf"$v_{{\rm proj}}$ [{velocity.get_unit}]")
    plt.show()

inclination_widget = widgets.FloatSlider(value = 40,
                          min = -90, 
                          max = 90,
                          step = 1.0,
                          description = "inclination",
                          continuous_update = False)

vshift_widget = widgets.FloatSlider(value = 0,
                          min = -vshift_extent, 
                          max = vshift_extent,
                          step = 0.1,
                          description = "vshift",
                          continuous_update = False)

interact(plot_vmap, inclination=inclination_widget, vshift=vshift_widget)

interactive(children=(FloatSlider(value=40.0, continuous_update=False, description='inclination', max=90.0, mi…

<function __main__.plot_vmap(inclination, vshift)>